# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.  
 
Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import pandas as pd 

df = pd.read_csv(r'C:/Users/Sebas!/Documents/GitHub/sentiments.csv',encoding='latin-1')
df.columns= ['target','id','date','flag','user','text']

In [2]:
df['text'][0]

"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
# your code here
from cleaning_functions import clean_up, tokenize, stem_and_lemmatize, remove_stopwords

sample = df.sample(n=25000, replace=False, random_state=3)

txt_processed = []

for text in sample['text']:
    cleaned = clean_up(text)
    token = tokenize(cleaned)
    stemAndlemma = stem_and_lemmatize(token)
    filtered = remove_stopwords(stemAndlemma)    
    txt_processed.append(filtered)

sample['text_processed'] = txt_processed



### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [71]:
# your code here
from nltk.probability import ConditionalFreqDist
import nltk
import random

all_words = []
for lst in sample['text_processed']:
     for word in lst:
         all_words.append(word)

all_words = nltk.FreqDist(all_words)


bag_words = list(all_words.keys())[:5000]


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [16]:
sample



,target,id,date,flag,user,text,text_processed
1164659,4,1979779549,Sun May 31 04:28:28 PDT 2009,NO_QUERY,MarcyChen,Informal study shows that my followers are coo...,"[inform, studi, show, my, follow, cooler, foll..."
228999,0,1978383347,Sat May 30 23:11:37 PDT 2009,NO_QUERY,emizell,i lost 2 followers...what did i do wrong??,"[lost, follow, did, do, wrong]"
1502596,4,2071730438,Sun Jun 07 19:50:23 PDT 2009,NO_QUERY,Jen_Kirby,Excited that I get to go into work late tomorrow,"[excit, i, get, go, work, late, tomorrow]"
950813,4,1824089618,Sun May 17 00:00:32 PDT 2009,NO_QUERY,Zombulator,@neolee23 Sorry for my late reply! I saw it on...,"[sorri, my, late, repli, saw, on, youtub, didn..."
1258437,4,1997910291,Mon Jun 01 17:40:09 PDT 2009,NO_QUERY,babybee3,bow check a wow wow! i had a good day!! mike t...,"[bow, check, wow, wow, had, good, day, mike, h..."
...,...,...,...,...,...,...,...
1023092,4,1882933753,Fri May 22 07:44:00 PDT 2009,NO_QUERY,YllwCkeNoFrstng,@tomogirl79 haha I love how you're surprised t...,"[haha, love, you, surpris, you, rub, on]"
1398541,4,2054113019,Sat Jun 06 06:35:23 PDT 2009,NO_QUERY,pointee,immm doing good! meeting taylon / pablo at the...,"[immm, good, meet, taylon, pablo, the, park, t..."
1458039,4,2063635277,Sun Jun 07 03:28:05 PDT 2009,NO_QUERY,anarmnetwork,@chempaka otw? hati2..,"[otw, hati]"
217219,0,1975887217,Sat May 30 16:07:01 PDT 2009,NO_QUERY,finnern,"You know that your kid is not feeling well, wh...","[know, your, kid, not, feel, well, ,, she, t, ..."


In [74]:
# your code here
dic_feat = {}

words = set(word for lst in sample['text_processed'] for word in lst)

for word in bag_words:
    dic_feat[word] = (word in words)

features = [({keys:dic_feat[keys]}, dic_feat[keys]) for keys in dic_feat.keys()]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [82]:
# your code here
training = features[:3000]
test = features[3000:]

classifier = nltk.NaiveBayesClassifier.train(training)

In [83]:
classifier.show_most_informative_features()

Most Informative Features


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [84]:
# your code here
print(nltk.classify.accuracy(classifier,test))
classifier.show_most_informative_features(15)

1.0
Most Informative Features


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [10]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [11]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [12]:
# your code here